<a href="https://colab.research.google.com/github/jungminchoi123/ECE539/blob/main/activity07_starter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
# from google.colab import drive
import numpy as np

# makes printing more human-friendly
np.set_printoptions(precision=3,suppress=True)

In [45]:
# a) Load data

colab = False  # Set to True if using colab
if colab:
    # May require changing paths to file
    drive.mount('/content/drive')
    with open('/content/drive/activity07_data.csv', 'r') as f:
      data = np.genfromtxt(f,delimiter=',')
else:
    # May require changing paths to file
    with open('/content/drive/activity07_data.csv', 'r', encoding='utf-8-sig') as f:
      data = np.genfromtxt(f,delimiter=',')

X = data[:,:-1]
y = data[:,-1]

In [58]:
# b) number of samples, features dimension, the number of classes

num_samples = X.shape[0]
num_feats = X.shape[1]
num_classes = len(np.unique(y))
unique_classes, counts = np.unique(y, return_counts=True)
num_samples_per_class = dict(zip(unique_classes, counts))

print(f'num of samples: {num_samples}')
print(f'num of feature dimensions: {num_feats}')
print(f'num of classes: {num_classes}')
for cls in num_samples_per_class:
  print(f'class {cls} has {num_samples_per_class[cls]} samples')

num of samples: 178
num of feature dimensions: 13
num of classes: 3
class 1.0 has 59 samples
class 2.0 has 71 samples
class 3.0 has 48 samples


In [47]:
# c) check nan, data imputation
from sklearn.impute import KNNImputer

if np.sum(np.isnan(X)):
  print('Total of NaN before imputation:', np.sum(np.isnan(X)))
  X = KNNImputer(n_neighbors=2, weights="uniform").fit_transform(X)
  print('Total of NaN after imputation:', np.sum(np.isnan(X)))
else:
  print('no NaN')

Total of NaN before imputation: 5
Total of NaN after imputation: 0


#### Q) How are the missing values completed when using

---

KNNImputer?
A) Answer here

In [48]:
# d) partition 80/20
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print('training data size: ', X_train.shape[0])
print('testing data size: ', X_test.shape[0])

training data size:  142
testing data size:  36


In [53]:
# e) standardize to -5 to 5

X_train_min = np.min(X_train, axis=0)
X_train_max = np.max(X_train, axis=0)
X_train_standardized = 10 * (X_train - X_train_min) / (X_train_max - X_train_min) - 5
print('min training data in each dimension, after standardization:', np.min(X_train_standardized, axis=0))
print('max training data in each dimension, after standardization:', np.max(X_train_standardized, axis=0))

# Warning: When standardizing the test set, we should use statistics like min or max computed from the training set.
X_test_standardized = 10 * (X_test - X_train_min) / (X_train_max - X_train_min) - 5
print('min testing data in each dimension, after standardization:', np.min(X_test_standardized, axis=0))
print('max testing data in each dimension, after standardization:', np.max(X_test_standardized, axis=0))

min training data in each dimension, after standardization: [-5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5.]
max training data in each dimension, after standardization: [5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5.]
min testing data in each dimension, after standardization: [-4.    -5.305 -3.182 -2.732 -4.13  -3.724 -4.662 -4.245 -5.032 -5.409
 -4.268 -4.89  -4.63 ]
max testing data in each dimension, after standardization: [3.605 2.556 3.021 2.423 0.761 3.    2.574 4.434 1.551 2.247 1.748 4.341
 6.048]


In [50]:
# f) standardize to 0 mean, unit variance

X_train_mean = np.mean(X_train, axis=0)
X_train_std = np.std(X_train, axis=0)
X_train_standardized = (X_train - X_train_mean) / X_train_std
print('mean training data in each dimension, after standardization:', np.mean(X_train_standardized, axis=0))
print('std training data in each dimension, after standardization:', np.std(X_train_standardized, axis=0))

# Warning: When standardizing the test set, we should use statistics like min or max computed from the training set.
X_test_standardized = (X_test - X_train_mean) / X_train_std
print('mean testing data in each dimension, after standardization:', np.mean(X_test_standardized, axis=0))
print('std testing data in each dimension, after standardization:', np.std(X_test_standardized, axis=0))

mean training data in each dimension, after standardization: [ 0. -0.  0.  0. -0.  0.  0. -0.  0.  0.  0.  0. -0.]
std training data in each dimension, after standardization: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
mean testing data in each dimension, after standardization: [ 0.13  -0.161  0.101  0.033 -0.238  0.047  0.127 -0.232 -0.153  0.001
  0.023  0.13   0.197]
std testing data in each dimension, after standardization: [0.945 0.87  0.91  0.825 0.846 0.907 0.977 0.823 0.9   0.977 0.881 0.91
 1.182]


In [54]:
# g) k fold
from sklearn.model_selection import KFold
import numpy as np

kf = KFold(n_splits=3)

# Count the class distributions in each partition
for train_index, val_index in kf.split(X, y):
  print('training class distribution: ', np.unique(y[train_index], return_counts=True))
  print('validation class distribution: ', np.unique(y[val_index], return_counts=True))

training class distribution:  (array([2., 3.]), array([70, 48]))
validation class distribution:  (array([1., 2.]), array([59,  1]))
training class distribution:  (array([1., 2., 3.]), array([59, 12, 48]))
validation class distribution:  (array([2.]), array([59]))
training class distribution:  (array([1., 2.]), array([59, 60]))
validation class distribution:  (array([2., 3.]), array([11, 48]))


In [52]:
if colab:
    drive.flush_and_unmount()